In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
import time
import matplotlib.pyplot as plt

In [2]:
import os
import sys

# Añadimos la carpeta 'drive' al path
ruta_carpeta_drive = os.path.abspath('../drive') #.. era para salirse de la carpeta actual y entrar en la de drive
if ruta_carpeta_drive not in sys.path:
    sys.path.insert(0, ruta_carpeta_drive)

import drive

# Descargamos los datos en formato parquet de Google Drive
# guardar localmente
drive.descargar_archivos_concretos('DATOS_MIN.parquet', '../drive')
#hay que pasarle la ruta a la carpeta drive desde donde lo estamos ejecutando
# y el nombre del archivo que hemos puesto en el archivos_info.txt (MLP_runs.db en nuestro ejemplo, en vez de datosEntrenamiento.parquet...)
# leer para utilizar esto ya en el modelo
datos = pd.read_parquet('datos/DATOS_MIN.parquet') # 

Archivo DATOS_MIN.parquet guardado en: ./datos/DATOS_MIN.parquet


In [3]:
datos

,name,EventStartTime,SalesStartTIme,SalesEndTime,min_price,nameArtist,VenueName,VenueCity,VenueState,num_links,...,Rock,Singer-Songwriter,Soul,Urban,World,tiene_wiki,tiene_youtube,dias_Venta,dias_desdeFinVenta,num_noches
0,Jamie Cullum,2024-06-14 20:00:00+00:00,2023-11-30 16:56:00+00:00,2024-06-12 21:59:00+00:00,30.01,Jamie Cullum,Auditorio Starlite,Marbella,Málaga,9,...,0,0,0,0,0,1,1,195,1,1
1,Sheryl Crow,2024-06-21 20:00:00+00:00,2023-11-28 20:57:00+00:00,2024-06-19 21:59:00+00:00,30.01,Sheryl Crow,Auditorio Starlite,Marbella,Málaga,9,...,1,0,0,0,0,1,1,204,1,1
2,Aitana,2024-06-22 20:00:00+00:00,2024-01-09 12:29:00+00:00,2024-06-20 21:59:00+00:00,65.18,Aitana,Auditorio Starlite,Marbella,Málaga,5,...,0,0,0,0,0,0,1,163,1,1
3,Christian Nodal,2024-07-13 20:00:00+00:00,2023-11-28 21:00:00+00:00,2024-07-11 21:59:00+00:00,30.01,Christian Nodal,Auditorio Starlite,Marbella,Málaga,8,...,0,0,0,0,1,0,1,226,1,1
4,Take That,2024-07-14 20:00:00+00:00,2023-11-28 21:00:00+00:00,2024-07-12 21:59:00+00:00,55.36,Robbie Williams,Auditorio Starlite,Marbella,Málaga,10,...,1,0,0,0,0,1,1,227,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Abba The New Experience & Queen Forever,2024-05-29 18:00:00+00:00,2024-02-28 14:00:00+00:00,2024-05-29 18:00:00+00:00,20.00,Abba The New Experience,Zona Camping Circuit Catalunya,Montmeló,Barcelona,0,...,1,0,0,0,0,0,0,91,0,1
490,Front 242,2024-10-13 18:30:00+00:00,2024-03-08 09:00:00+00:00,2024-10-13 18:30:00+00:00,33.13,Front 242,Sala Apolo,Barcelona,Barcelona,9,...,1,0,0,0,0,1,1,219,0,1
491,Front 242,2024-10-14 19:00:00+00:00,2024-03-08 09:00:00+00:00,2024-10-14 19:00:00+00:00,33.13,Front 242,Sala La Paqui (Antigua Sala But),Madrid,Madrid,9,...,1,0,0,0,0,1,1,220,0,1
492,Garbage,2024-10-16 18:30:00+00:00,2024-03-08 09:00:00+00:00,2024-10-16 18:30:00+00:00,40.00,Garbage,Sala Razzmatazz 1,Barcelona,Barcelona,10,...,1,0,0,0,0,1,1,222,0,1


In [5]:
 # Eliminamos las categoricas
columnas=['name','EventStartTime', 'SalesStartTIme', 'SalesEndTime', 'nameArtist', 'VenueName',
          'VenueCity', 'VenueState','Generos_combinados']

datos.drop(columnas, axis=1, inplace=True)

In [4]:
from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 83 #fijamos la semilla
# y es la variable objetivo
X = datos.iloc[:,1:]
y = datos['min_price']

folds = datos['fold'].unique()


KeyError: 'fold'

In [8]:
# Sets the sqlite db as the MLFLOW_TRACKING_URI 
import os
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# WARNING: TO SEE THE LOCAL SERVER YOU HAVE TO CHOOSE THE CORRECT BACKEND STORE AS FOLLOWS:
# mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import time
import numpy as np
import mlflow
import mlflow.sklearn

# Iniciar el experimento de MLflow
mlflow.set_experiment("MinPriceDecisionTree")

# Iniciar el run de MLflow
with mlflow.start_run(run_name='DecisionTreeRegressor'):
    # Tu código aquí
    folds = datos['fold'].unique()

    tree = DecisionTreeRegressor(random_state=RANDOM_STATE)

    params = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10,20,15],
        'min_samples_leaf': [1, 2, 4,6,5]
    }

    start_time_grid = time.time()
    gridSearch = GridSearchCV(estimator=tree, param_grid=params, n_jobs=-1, scoring='neg_mean_squared_error')
    gridSearch.fit(X, y)
    end_time_grid = time.time()
    execution_time_grid = end_time_grid - start_time_grid
    print("Tiempo de ejecución gridSearch:", execution_time_grid, "segundos")

    # Guardar el modelo con MLflow
    mlflow.sklearn.log_model(gridSearch.best_estimator_, "decision_tree_regressor_model")

    print('Mejores parámetros gridSearch: ', gridSearch.best_params_)
    print('Mejor puntuación (MSE): ', -gridSearch.best_score_)

    best_model = gridSearch.best_estimator_

    mape_train_list = []
    mape_val_list = []
    rmse_train_list = []
    rmse_val_list = []

    start_time_cross_val = time.time()

    for fold in folds:
        X_train = datos[datos['fold'] != fold].drop(['fold', 'min_price'], axis=1)
        X_val = datos[datos['fold'] == fold].drop(['fold', 'min_price'], axis=1)
        y_train = datos[datos['fold'] != fold]['min_price']
        y_val = datos[datos['fold'] == fold]['min_price']

        best_model.fit(X_train, y_train)

        predicciones_train = best_model.predict(X_train)
        predicciones_val = best_model.predict(X_val)

        mape_train = mean_absolute_percentage_error(y_train, predicciones_train)
        mape_val = mean_absolute_percentage_error(y_val, predicciones_val)

        rmse_train = mean_squared_error(y_train, predicciones_train, squared=False)
        rmse_val = mean_squared_error(y_val, predicciones_val, squared=False)

        mape_train_list.append(mape_train)
        mape_val_list.append(mape_val)
        rmse_train_list.append(rmse_train)
        rmse_val_list.append(rmse_val)

        print(f"Fold {fold}: MAPE Train: {mape_train}, MAPE Val: {mape_val}, RMSE Train: {rmse_train}, RMSE Val: {rmse_val}")

    end_time_cross_val = time.time()
    execution_time_cross_val = end_time_cross_val - start_time_cross_val
    print("Tiempo de ejecución validación cruzada:", execution_time_cross_val, "segundos")

    print("MAPE Train Promedio:", np.mean(mape_train_list))
    print("MAPE Val Promedio:", np.mean(mape_val_list))
    print("RMSE Train Promedio:", np.mean(rmse_train_list))
    print("RMSE Val Promedio:", np.mean(rmse_val_list))

    # Guardar métricas con MLflow
    mlflow.log_metric("MAPE Train Promedio", np.mean(mape_train_list))
    mlflow.log_metric("MAPE Val Promedio", np.mean(mape_val_list))
    mlflow.log_metric("RMSE Train Promedio", np.mean(rmse_train_list))
    mlflow.log_metric("RMSE Val Promedio", np.mean(rmse_val_list))

    # Guardar tiempo de ejecución con MLflow
    mlflow.log_metric("Tiempo de ejecución gridSearch", execution_time_grid)
    mlflow.log_metric("Tiempo de ejecución validación cruzada", execution_time_cross_val)

    # Finalizar run de MLflow
    mlflow.end_run()


2024/05/12 20:03:26 INFO mlflow.tracking.fluent: Experiment with name 'MinPriceDecisionTree' does not exist. Creating a new experiment.


KeyError: 'fold'

In [ ]:
# Obtener variables importantes
feature_importance = best_model.feature_importances_

# Obtener los índices de las variables más importantes
top_indices = np.argsort(feature_importance)[::-1][:10]

print("\nLas 10 variables más importantes:")
for i in top_indices:
    print(f"{X_train.columns[i]}: {feature_importance[i]}")



Las 10 variables más importantes:
seguidoresSpotify: 0.562010085964474
conciertos_del_artista_en_ciudad: 0.12870926297516308
NumGeneros: 0.08437993655023292
subsYT: 0.0492897809032739
Latin: 0.040637845860815966
peso_promotor: 0.039597147766925384
K-Pop: 0.031855182497007084
num_links: 0.02916637278631562
VenueClass: 0.014927615173626358
popularidad: 0.004975844850967514


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import time
import numpy as np

folds = datos['fold'].unique()

tree = DecisionTreeRegressor(random_state=RANDOM_STATE)

params = {
    'max_depth': [None, 15, 8, 18, 25, 35],
    'min_samples_split': [30, 40, 35, 3, 2],
    'min_samples_leaf': [2, 5, 4, 6, 3]
}

start_time_grid = time.time()
gridSearch = GridSearchCV(estimator=tree, param_grid=params, n_jobs=-1, scoring='neg_mean_squared_error')
gridSearch.fit(X, y)
end_time_grid = time.time()
execution_time_grid = end_time_grid - start_time_grid
print("Tiempo de ejecución gridSearch:", execution_time_grid, "segundos")

print('Mejores parámetros gridSearch: ', gridSearch.best_params_)
print('Mejor puntuación (MSE): ', -gridSearch.best_score_)

best_model = gridSearch.best_estimator_

mape_train_list = []
mape_val_list = []

start_time_cross_val = time.time()

for fold in folds:
    X_train = datos[datos['fold'] != fold].drop(['fold', 'min_price'], axis=1)
    X_val = datos[datos['fold'] == fold].drop(['fold', 'min_price'], axis=1)
    y_train = datos[datos['fold'] != fold]['min_price']
    y_val = datos[datos['fold'] == fold]['min_price']

    best_model.fit(X_train, y_train)

    predicciones_train = best_model.predict(X_train)
    predicciones_val = best_model.predict(X_val)

    mape_train = mean_absolute_percentage_error(y_train, predicciones_train)
    mape_val = mean_absolute_percentage_error(y_val, predicciones_val)

    mape_train_list.append(mape_train)
    mape_val_list.append(mape_val)

    print(f"Fold {fold}: MAPE Train: {mape_train}, MAPE Val: {mape_val}")

end_time_cross_val = time.time()
execution_time_cross_val = end_time_cross_val - start_time_cross_val
print("Tiempo de ejecución validación cruzada:", execution_time_cross_val, "segundos")

print("MAPE Train Promedio:", np.mean(mape_train_list))
print("MAPE Val Promedio:", np.mean(mape_val_list))


Tiempo de ejecución gridSearch: 4.591616868972778 segundos
Mejores parámetros gridSearch:  {'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 30}
Mejor puntuación (MSE):  232.35851414089012
Fold 0: MAPE Train: 0.1736278143825403, MAPE Val: 0.27509540610836647
Fold 1: MAPE Train: 0.17324540130898994, MAPE Val: 0.2417163887032596
Fold 2: MAPE Train: 0.18064054542443894, MAPE Val: 0.23001325786893953
Fold 3: MAPE Train: 0.16030064089308133, MAPE Val: 0.26157096043307987
Tiempo de ejecución validación cruzada: 0.053760528564453125 segundos
MAPE Train Promedio: 0.1719536005022626
MAPE Val Promedio: 0.2520990032784114


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import time
import numpy as np

folds = datos['fold'].unique()

# Modelo base sin hiperparámetros
base_model = DecisionTreeRegressor()

start_time_grid = time.time()
gridSearch = GridSearchCV(estimator=base_model, param_grid={}, n_jobs=-1, scoring='neg_mean_squared_error')
gridSearch.fit(X, y)
end_time_grid = time.time()
execution_time_grid = end_time_grid - start_time_grid
print("Tiempo de ejecución gridSearch:", execution_time_grid, "segundos")

print('Mejores parámetros gridSearch: ', gridSearch.best_params_)
print('Mejor puntuación (MSE): ', -gridSearch.best_score_)

best_model = gridSearch.best_estimator_

mape_train_list = []
mape_val_list = []
rmse_train_list = []
rmse_val_list = []

start_time_cross_val = time.time()

for fold in folds:
    X_train = datos[datos['fold'] != fold].drop(['fold', 'min_price'], axis=1)
    X_val = datos[datos['fold'] == fold].drop(['fold', 'min_price'], axis=1)
    y_train = datos[datos['fold'] != fold]['min_price']
    y_val = datos[datos['fold'] == fold]['min_price']

    best_model.fit(X_train, y_train)

    predicciones_train = best_model.predict(X_train)
    predicciones_val = best_model.predict(X_val)

    mape_train = mean_absolute_percentage_error(y_train, predicciones_train)
    mape_val = mean_absolute_percentage_error(y_val, predicciones_val)

    rmse_train = np.sqrt(mean_squared_error(y_train, predicciones_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, predicciones_val))

    mape_train_list.append(mape_train)
    mape_val_list.append(mape_val)
    rmse_train_list.append(rmse_train)
    rmse_val_list.append(rmse_val)

    print(f"Fold {fold}: MAPE Train: {mape_train}, MAPE Val: {mape_val}")
    print(f"Fold {fold}: RMSE Train: {rmse_train}, RMSE Val: {rmse_val}")

end_time_cross_val = time.time()
execution_time_cross_val = end_time_cross_val - start_time_cross_val
print("Tiempo de ejecución validación cruzada:", execution_time_cross_val, "segundos")

print("MAPE Train Promedio:", np.mean(mape_train_list))
print("MAPE Val Promedio:", np.mean(mape_val_list))
print("RMSE Train Promedio:", np.mean(rmse_train_list))
print("RMSE Val Promedio:", np.mean(rmse_val_list))


Tiempo de ejecución gridSearch: 0.09241533279418945 segundos
Mejores parámetros gridSearch:  {}
Mejor puntuación (MSE):  342.5242719109462
Fold 0: MAPE Train: 0.0, MAPE Val: 0.37225341495042297
Fold 0: RMSE Train: 0.0, RMSE Val: 24.58039716928919
Fold 1: MAPE Train: 0.0020444510444510445, MAPE Val: 0.2981460474149125
Fold 1: RMSE Train: 0.8120710926141008, RMSE Val: 13.53583612775433
Fold 2: MAPE Train: 0.0020389403947355432, MAPE Val: 0.26866616962665635
Fold 2: RMSE Train: 0.8109759186935261, RMSE Val: 22.71608151397581
Fold 3: MAPE Train: 0.0020389403947355432, MAPE Val: 0.2973215814365331
Fold 3: RMSE Train: 0.8109759186935261, RMSE Val: 18.24208499707888
Tiempo de ejecución validación cruzada: 0.09334897994995117 segundos
MAPE Train Promedio: 0.0015305829584805327
MAPE Val Promedio: 0.30909680335713124
RMSE Train Promedio: 0.6085057325002883
RMSE Val Promedio: 19.76859995202455


------------- Random Search -------------

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import time
import numpy as np

folds = datos['fold'].unique()

# Modelo base sin hiperparámetros
base_model = DecisionTreeRegressor()

start_time_grid = time.time()
gridSearch = GridSearchCV(estimator=base_model, param_grid={}, n_jobs=-1, scoring='neg_mean_squared_error')
gridSearch.fit(X, y)
end_time_grid = time.time()
execution_time_grid = end_time_grid - start_time_grid
print("Tiempo de ejecución gridSearch:", execution_time_grid, "segundos")

print('Mejores parámetros gridSearch: ', gridSearch.best_params_)
print('Mejor puntuación (MSE): ', -gridSearch.best_score_)

best_model = gridSearch.best_estimator_

mape_train_list = []
mape_val_list = []
rmse_train_list = []
rmse_val_list = []

start_time_cross_val = time.time()

for fold in folds:
    X_train = datos[datos['fold'] != fold].drop(['fold', 'min_price'], axis=1)
    X_val = datos[datos['fold'] == fold].drop(['fold', 'min_price'], axis=1)
    y_train = datos[datos['fold'] != fold]['min_price']
    y_val = datos[datos['fold'] == fold]['min_price']

    best_model.fit(X_train, y_train)

    predicciones_train = best_model.predict(X_train)
    predicciones_val = best_model.predict(X_val)

    mape_train = mean_absolute_percentage_error(y_train, predicciones_train)
    mape_val = mean_absolute_percentage_error(y_val, predicciones_val)

    rmse_train = np.sqrt(mean_squared_error(y_train, predicciones_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, predicciones_val))

    mape_train_list.append(mape_train)
    mape_val_list.append(mape_val)
    rmse_train_list.append(rmse_train)
    rmse_val_list.append(rmse_val)

    print(f"Fold {fold}: MAPE Train: {mape_train}, MAPE Val: {mape_val}")
    print(f"Fold {fold}: RMSE Train: {rmse_train}, RMSE Val: {rmse_val}")

end_time_cross_val = time.time()
execution_time_cross_val = end_time_cross_val - start_time_cross_val
print("Tiempo de ejecución validación cruzada:", execution_time_cross_val, "segundos")

print("MAPE Train Promedio:", np.mean(mape_train_list))
print("MAPE Val Promedio:", np.mean(mape_val_list))
print("RMSE Train Promedio:", np.mean(rmse_train_list))
print("RMSE Val Promedio:", np.mean(rmse_val_list))


Tiempo de ejecución gridSearch: 3.856738805770874 segundos
Mejores parámetros gridSearch:  {}
Mejor puntuación (MSE):  358.8220183323026
Fold 0: MAPE Train: 0.0, MAPE Val: 0.3766667419807861
Fold 0: RMSE Train: 0.0, RMSE Val: 26.397612092641037
Fold 1: MAPE Train: 0.0020444510444510445, MAPE Val: 0.2921343160484902
Fold 1: RMSE Train: 0.8120710926141008, RMSE Val: 13.025902915890073
Fold 2: MAPE Train: 0.0020389403947355454, MAPE Val: 0.23565293388850772
Fold 2: RMSE Train: 0.8109759186935261, RMSE Val: 22.630819136671743
Fold 3: MAPE Train: 0.0020389403947355432, MAPE Val: 0.3298709977457101
Fold 3: RMSE Train: 0.8109759186935261, RMSE Val: 20.027807984972466
Tiempo de ejecución validación cruzada: 0.07564568519592285 segundos
MAPE Train Promedio: 0.0015305829584805334
MAPE Val Promedio: 0.3085812474158735
RMSE Train Promedio: 0.6085057325002883
RMSE Val Promedio: 20.520535532543832


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import time
import numpy as np

folds = datos['fold'].unique()

params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 20, 30],
    'min_samples_leaf': [1, 2, 4, 6, 8]
}

base_model = RandomForestRegressor(random_state=RANDOM_STATE)

start_time_grid = time.time()
gridSearch = GridSearchCV(estimator=base_model, param_grid=params, n_jobs=-1, scoring='neg_mean_squared_error')
gridSearch.fit(X, y)
end_time_grid = time.time()
execution_time_grid = end_time_grid - start_time_grid
print("Tiempo de ejecución gridSearch:", execution_time_grid, "segundos")

print('Mejores parámetros gridSearch: ', gridSearch.best_params_)
print('Mejor puntuación (MSE): ', -gridSearch.best_score_)

best_model = gridSearch.best_estimator_

mape_train_list = []
mape_val_list = []
rmse_train_list = []
rmse_val_list = []

start_time_cross_val = time.time()

for fold in folds:
    X_train = datos[datos['fold'] != fold].drop(['fold', 'min_price'], axis=1)
    X_val = datos[datos['fold'] == fold].drop(['fold', 'min_price'], axis=1)
    y_train = datos[datos['fold'] != fold]['min_price']
    y_val = datos[datos['fold'] == fold]['min_price']

    best_model.fit(X_train, y_train)

    predicciones_train = best_model.predict(X_train)
    predicciones_val = best_model.predict(X_val)

    mape_train = mean_absolute_percentage_error(y_train, predicciones_train)
    mape_val = mean_absolute_percentage_error(y_val, predicciones_val)

    rmse_train = np.sqrt(mean_squared_error(y_train, predicciones_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, predicciones_val))

    mape_train_list.append(mape_train)
    mape_val_list.append(mape_val)
    rmse_train_list.append(rmse_train)
    rmse_val_list.append(rmse_val)

    print(f"Fold {fold}: MAPE Train: {mape_train}, MAPE Val: {mape_val}")
    print(f"Fold {fold}: RMSE Train: {rmse_train}, RMSE Val: {rmse_val}")

end_time_cross_val = time.time()
execution_time_cross_val = end_time_cross_val - start_time_cross_val
print("Tiempo de ejecución validación cruzada:", execution_time_cross_val, "segundos")

print("MAPE Train Promedio:", np.mean(mape_train_list))
print("MAPE Val Promedio:", np.mean(mape_val_list))
print("RMSE Train Promedio:", np.mean(rmse_train_list))
print("RMSE Val Promedio:", np.mean(rmse_val_list))


Tiempo de ejecución gridSearch: 531.0662043094635 segundos
Mejores parámetros gridSearch:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}
Mejor puntuación (MSE):  193.14704499095032
Fold 0: MAPE Train: 0.10677641922741977, MAPE Val: 0.2599559564953453
Fold 0: RMSE Train: 7.411134272224323, RMSE Val: 13.45500131331651
Fold 1: MAPE Train: 0.10645370963373743, MAPE Val: 0.2566491841784534
Fold 1: RMSE Train: 7.424120338722791, RMSE Val: 10.079492495994902
Fold 2: MAPE Train: 0.11555279238017838, MAPE Val: 0.22303226868537707
Fold 2: RMSE Train: 6.723513705610611, RMSE Val: 19.051681488218843
Fold 3: MAPE Train: 0.09428845739672388, MAPE Val: 0.24807342678625563
Fold 3: RMSE Train: 6.692409187723524, RMSE Val: 13.479901167068967
Tiempo de ejecución validación cruzada: 0.899580717086792 segundos
MAPE Train Promedio: 0.10576784465951486
MAPE Val Promedio: 0.24692770903635786
RMSE Train Promedio: 7.062794376070312
RMSE Val Promedio: 14.016519116149807


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import time
import numpy as np

folds = datos['fold'].unique()

params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 20, 25, 30, 45, 55],
    'min_samples_split': [3, 5, 15, 20, 30],
    'min_samples_leaf': [1, 3, 5, 6, 8]
}

base_model = RandomForestRegressor(random_state=RANDOM_STATE)

start_time_random = time.time()
random_search = RandomizedSearchCV(estimator=base_model, param_distributions=params, n_iter=10, random_state=RANDOM_STATE, n_jobs=-1, scoring='neg_mean_squared_error')
random_search.fit(X, y)
end_time_random = time.time()
execution_time_random = end_time_random - start_time_random
print("Tiempo de ejecución RandomizedSearch:", execution_time_random, "segundos")

print('Mejores parámetros RandomizedSearch: ', random_search.best_params_)
print('Mejor puntuación (MSE): ', -random_search.best_score_)

best_model = random_search.best_estimator_

mape_train_list = []
mape_val_list = []
rmse_train_list = []
rmse_val_list = []

start_time_cross_val = time.time()

for fold in folds:
    X_train = datos[datos['fold'] != fold].drop(['fold', 'min_price'], axis=1)
    X_val = datos[datos['fold'] == fold].drop(['fold', 'min_price'], axis=1)
    y_train = datos[datos['fold'] != fold]['min_price']
    y_val = datos[datos['fold'] == fold]['min_price']

    best_model.fit(X_train, y_train)

    predicciones_train = best_model.predict(X_train)
    predicciones_val = best_model.predict(X_val)

    mape_train = mean_absolute_percentage_error(y_train, predicciones_train)
    mape_val = mean_absolute_percentage_error(y_val, predicciones_val)

    rmse_train = np.sqrt(mean_squared_error(y_train, predicciones_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, predicciones_val))

    mape_train_list.append(mape_train)
    mape_val_list.append(mape_val)
    rmse_train_list.append(rmse_train)
    rmse_val_list.append(rmse_val)

    print(f"Fold {fold}: MAPE Train: {mape_train}, MAPE Val: {mape_val}")
    print(f"Fold {fold}: RMSE Train: {rmse_train}, RMSE Val: {rmse_val}")

end_time_cross_val = time.time()
execution_time_cross_val = end_time_cross_val - start_time_cross_val
print("Tiempo de ejecución validación cruzada:", execution_time_cross_val, "segundos")

print("MAPE Train Promedio:", np.mean(mape_train_list))
print("MAPE Val Promedio:", np.mean(mape_val_list))
print("RMSE Train Promedio:", np.mean(rmse_train_list))
print("RMSE Val Promedio:", np.mean(rmse_val_list))


Tiempo de ejecución RandomizedSearch: 11.838037967681885 segundos
Mejores parámetros RandomizedSearch:  {'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}
Mejor puntuación (MSE):  200.16835409821948
Fold 0: MAPE Train: 0.07469014072088348, MAPE Val: 0.24697462240269843
Fold 0: RMSE Train: 6.172274027380235, RMSE Val: 13.456159201622986
Fold 1: MAPE Train: 0.07893136940665736, MAPE Val: 0.24698586113937981
Fold 1: RMSE Train: 5.985549591592963, RMSE Val: 9.802985625204272
Fold 2: MAPE Train: 0.08593192494187306, MAPE Val: 0.21613715838584943
Fold 2: RMSE Train: 5.431341360197993, RMSE Val: 19.163071106514863
Fold 3: MAPE Train: 0.07054660386815749, MAPE Val: 0.24836492090447543
Fold 3: RMSE Train: 5.577307954345067, RMSE Val: 14.01509158446926
Tiempo de ejecución validación cruzada: 2.1803009510040283 segundos
MAPE Train Promedio: 0.07752500973439284
MAPE Val Promedio: 0.23961564070810076
RMSE Train Promedio: 5.791618233379064
RMSE Val Promedio: 14.109

In [ ]:
import matplotlib.pyplot as plt

# Obtener los resultados de la búsqueda aleatoria de hiperparámetros
results = random_search.cv_results_

# Obtener los valores probados para 'n_estimators'
n_estimators = params['n_estimators']

# Obtener los puntajes de entrenamiento y prueba para cada valor de 'n_estimators'
train_scores_mean = -results['mean_train_score']
test_scores_mean = -results['mean_test_score']

# Dibujar las líneas
plt.plot(n_estimators, train_scores_mean, color="r",  label="Training")
plt.plot(n_estimators, test_scores_mean, color="g", label="Test")

# Crear el gráfico
plt.title("Curva de aprendizaje para 'n_estimators'")
plt.xlabel("n_estimators"), plt.ylabel("Mean Squared Error"), plt.legend(loc="best")
plt.tight_layout()
plt.show()


KeyError: 'mean_train_score'